In [ ]:
import DeepFMKit.core as dfm
from DeepFMKit.workers import calculate_ambiguity_boundary_point
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm import tqdm
import multiprocessing
import os

m_range=np.logspace(0, 3, 100)
dm_range=np.logspace(-5, 0, 100)
delta_f=np.logspace(9, 13, 100)

wavelength_nm=1550
n_cores=None

if n_cores is None:
    n_cores = os.cpu_count()
    
f0 = dfm.sc.c / (wavelength_nm * 1e-9)
    
print("=" * 60)
print("Generating Ambiguity Resolution Boundary Plot")
print(f"Carrier Frequency (f0): {f0 / 1e12:.1f} THz")
print("=" * 60)

jobs = []
for j, df in enumerate(df_range):
    for i, dm in enumerate(dm_range):
        jobs.append({
            'delta_f': df,
            'delta_m': dm,
            'f0': f0,
            'grid_i': i, 'grid_j': j
        })

grid_shape = (len(dm_range), len(df_range))
error_grid = np.zeros(grid_shape)

if __name__ == "__main__":
    with multiprocessing.Pool(processes=n_cores) as pool:
        results_iterator = pool.imap(calculate_ambiguity_boundary_point, jobs)
        for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Boundary Grid"):
            i, j, error = result
            error_grid[i, j] = error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Compute log10-transformed versions
log_df = np.log10(df_range)
log_dm = np.log10(dm_range)
log_error = np.log10(error_grid)

# Set up log-scale meshgrid
log_df_grid, log_dm_grid = np.meshgrid(log_df, log_dm)

# --- Plotting ---
fig, ax = plt.subplots(figsize=(12, 9))

# pcolormesh with original values and LogNorm (not log10)
im = ax.pcolormesh(
    df_range / 1e9, dm_range, error_grid,
    norm=LogNorm(vmin=np.pi/100, vmax=np.pi*100),
    cmap='magma_r', shading='auto'
)
fig.colorbar(im, ax=ax, label=r'Coarse Phase Error, $|\delta\Phi_{\rm coarse}|$ (rad)')

# Contour in fully log-log-linear space
CS = ax.contour(
    10**log_df_grid / 1e9, 10**log_dm_grid, log_error,
    levels=[np.log10(np.pi)], colors='cyan', linewidths=3
)
ax.clabel(CS, inline=True, fontsize=12,
          fmt=r'$|\delta\Phi_{\rm coarse}| = \pi$')

# Set log axes and labels
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Modulation Amplitude, $\Delta f$ (GHz)', fontsize=14)
ax.set_ylabel(r"Total Error on m, $|\delta m|$ (rad)", fontsize=14)
ax.set_title('Ambiguity Resolution Boundary', fontsize=16)

# Region labels
ax.text(0.5, 0.2, 'Ambiguity\nResolved', transform=ax.transAxes,
        ha='center', fontsize=15, color='white', weight='bold')
ax.text(0.5, 0.6, 'Fringe\nSlip', transform=ax.transAxes,
        ha='center', fontsize=15, color='white', weight='bold')

ax.grid(True, which='both', linestyle=':', alpha=0.5)
plt.show()